Test connection to local ChromaDB instance

In [81]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions

client = chromadb.PersistentClient(path="../components/vector_db")

In [82]:
client.heartbeat()

1766580221588380000

In [84]:
collection_name = "collection"

try:
    collection = client.create_collection(name=collection_name)
    print(f"Collection {collection_name} created")
except Exception as e:
    collection = client.get_collection(name=collection_name)
    print(f"Collection {collection_name} already exists")    

Collection collection already exists


Embedder

In [ ]:
voyageai_ef = embedding_functions.VoyageAIEmbeddingFunction(
    api_key="YOUR_API_KEY",  
    model_name="voyage-3-large"
)